In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.model_selection import cross_val_score,StratifiedKFold,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,precision_score,f1_score,recall_score

In [3]:
results = pd.read_csv(r'data\results.csv')
races = pd.read_csv(r'data\races.csv')
quali = pd.read_csv(r'data\qualifying.csv')
drivers = pd.read_csv(r'data\drivers.csv')
constructors = pd.read_csv(r'data\constructors.csv')
circuit = pd.read_csv(r'data\circuits.csv')

In [5]:
df1 = pd.merge(races,results,how='inner',on=['raceId'])
df2 = pd.merge(df1,quali,how='inner',on=['raceId','driverId','constructorId'])
df3 = pd.merge(df2,drivers,how='inner',on=['driverId'])
df4 = pd.merge(df3,constructors,how='inner',on=['constructorId'])
df5 = pd.merge(df4, circuit, how='inner', on=['circuitId'], suffixes=('_df4', '_circuit'))


In [9]:
print(df5.shape)
print(df5.columns)

(8403, 51)
Index(['raceId', 'year', 'round', 'circuitId', 'name_x', 'date', 'time_x',
       'url_x', 'resultId', 'driverId', 'constructorId', 'number_x', 'grid',
       'position_x', 'positionText', 'positionOrder', 'points', 'laps',
       'time_y', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'qualifyId', 'number_y', 'position_y',
       'q1', 'q2', 'q3', 'driverRef', 'number', 'code', 'forename', 'surname',
       'dob', 'nationality_x', 'url_y', 'constructorRef', 'name_y',
       'nationality_y', 'url_df4', 'circuitRef', 'name', 'location', 'country',
       'lat', 'lng', 'alt', 'url_circuit'],
      dtype='object')


In [16]:
# List of columns to drop
columns_to_drop = ['round','circuitId','time_x','url_x','resultId','driverId','constructorId','number_x','positionText','position_x','positionOrder','laps','time_y','rank','fastestLapTime','fastestLapSpeed','qualifyId','driverRef','number','code','url_y','circuitRef','location','lat','lng','alt','number_y','points','constructorRef','name_x','raceId','fastestLap','q2','q3','milliseconds','q1']
columns_to_drop = [col for col in columns_to_drop if col in df5.columns]

# Drop the columns
data = df5.drop(columns=columns_to_drop)


In [17]:
data.columns

Index(['year', 'date', 'grid', 'statusId', 'position_y', 'forename', 'surname',
       'dob', 'nationality_x', 'name_y', 'nationality_y', 'url_df4', 'name',
       'country', 'url_circuit'],
      dtype='object')

In [3]:
import F1PyStats as fp

In [15]:
data.columns

Index(['raceId', 'year', 'round', 'circuitId', 'name_x', 'date', 'time_x',
       'url_x', 'resultId', 'driverId', 'constructorId', 'number_x', 'grid',
       'position_x', 'positionText', 'positionOrder', 'points', 'laps',
       'time_y', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'qualifyId', 'number_y', 'position_y',
       'q1', 'q2', 'q3', 'driverRef', 'number', 'code', 'forename', 'surname',
       'dob', 'nationality_x', 'url_y', 'constructorRef', 'name_y',
       'nationality_y', 'url_df4', 'circuitRef', 'name', 'location', 'country',
       'lat', 'lng', 'alt', 'url_circuit'],
      dtype='object')

Season 1950 - Round 1: British Grand Prix at Silverstone Circuit
Race Results:
  Position 1: Nino Farina
  Position 2: Luigi Fagioli
  Position 3: Reg Parnell
  Position 4: Yves Cabantous
  Position 5: Louis Rosier
  Position 6: Bob Gerard
  Position 7: Cuth Harrison
  Position 8: Philippe Étancelin
  Position 9: David Hampshire
  Position 10: Brian Shawe Taylor
  Position 10: Joe Fry
  Position 11: Johnny Claes
  Position 12: Juan Fangio
  Position 13: Joe Kelly
  Position 14: Prince Bira
  Position 15: David Murray
  Position 16: Geoff Crossley
  Position 17: Toulo de Graffenried
  Position 18: Louis Chiron
  Position 19: Eugène Martin
  Position 20: Peter Walker
  Position 20: Tony Rolt
  Position 21: Leslie Johnson


Season 1950 - Round 2: Monaco Grand Prix at Circuit de Monaco
Race Results:
  Position 1: Juan Fangio
  Position 2: Alberto Ascari
  Position 3: Louis Chiron
  Position 4: Raymond Sommer
  Position 5: Prince Bira
  Position 6: Bob Gerard
  Position 7: Johnny Claes
  Po

In [2]:
import requests

# Example: Get race schedule for 2024 season
response = requests.get("http://ergast.com/api/f1/2023.json")
data = response.json()

# Print the race schedule
for race in data['MRData']['RaceTable']['Races']:
    print(f"Round {race['round']}: {race['raceName']} at {race['Circuit']['circuitName']}")


Round 1: Bahrain Grand Prix at Bahrain International Circuit
Round 2: Saudi Arabian Grand Prix at Jeddah Corniche Circuit
Round 3: Australian Grand Prix at Albert Park Grand Prix Circuit
Round 4: Azerbaijan Grand Prix at Baku City Circuit
Round 5: Miami Grand Prix at Miami International Autodrome
Round 6: Monaco Grand Prix at Circuit de Monaco
Round 7: Spanish Grand Prix at Circuit de Barcelona-Catalunya
Round 8: Canadian Grand Prix at Circuit Gilles Villeneuve
Round 9: Austrian Grand Prix at Red Bull Ring
Round 10: British Grand Prix at Silverstone Circuit
Round 11: Hungarian Grand Prix at Hungaroring
Round 12: Belgian Grand Prix at Circuit de Spa-Francorchamps
Round 13: Dutch Grand Prix at Circuit Park Zandvoort
Round 14: Italian Grand Prix at Autodromo Nazionale di Monza
Round 15: Singapore Grand Prix at Marina Bay Street Circuit
Round 16: Japanese Grand Prix at Suzuka Circuit
Round 17: Qatar Grand Prix at Losail International Circuit
Round 18: United States Grand Prix at Circuit of

In [4]:
import ergast_py

e = ergast_py.Ergast()

In [8]:
# http://ergast.com/api/f1/2008/5/results
race_results = e.season(2008).round(5).get_results()
print(race_results)

# http://ergast.com/api/f1/drivers/massa
felipe_massa = e.driver("massa").get_driver()
print(felipe_massa)

# http://ergast.com/api/f1/current/constructorStandings/3
constructor_standings = e.season().standing(3).get_constructor_standings()
print(constructor_standings)

[Race(season=2008, round_no=5, url='http://en.wikipedia.org/wiki/2008_Turkish_Grand_Prix', race_name='Turkish Grand Prix', circuit=Circuit(circuit_id='istanbul', url='http://en.wikipedia.org/wiki/Istanbul_Park', circuit_name='Istanbul Park', location=Location(latitude=40.9517, longitude=29.405, locality='Istanbul', country='Turkey')), date=datetime.datetime(2008, 5, 11, 12, 0, tzinfo=datetime.timezone.utc), results=[Result(number=2, position=1, position_text='1', points=10.0, driver=Driver(driver_id='massa', code='MAS', url='http://en.wikipedia.org/wiki/Felipe_Massa', given_name='Felipe', family_name='Massa', date_of_birth=datetime.date(1981, 4, 25), nationality='Brazilian', permanent_number=19), constructor=Constructor(constructor_id='ferrari', url='http://en.wikipedia.org/wiki/Scuderia_Ferrari', name='Ferrari', nationality='Italian'), grid=1, laps=58, status=1, time=Time(millis=datetime.time(1, 26, 49, 451000), time='1:26:49.451'), fastest_lap=FastestLap(rank=3, lap=16, time=datetime

In [12]:
from ergast_py.constants.status_type import StatusType
from ergast_py.models.circuit import Circuit
from ergast_py.models.constructor import Constructor
from ergast_py.models.driver import Driver
from ergast_py.models.race import Race
from ergast_py.models.season import Season
from ergast_py.models.standings_list import StandingsList
from ergast_py.models.status import Status
from ergast_py.requester import Requester
from ergast_py.type_constructor import TypeConstructor